In [ ]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip install keras_vggface
!pip install keras_application
!pip install keras_preprocessing
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0
!pip install keras==2.2.4

## Imports
import tensorflow as tf
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, NumpyArrayIterator
from keras.layers import Conv2D, Dense, BatchNormalization, Activation, Dropout, MaxPooling2D, Flatten
from keras.optimizers import Adam, RMSprop, SGD
from keras import regularizers
from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
import keras
from keras.models import Model
from google.colab import drive
drive.mount('/content/drive')
import datetime


  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-17bzxdnu
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-17bzxdnu
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp37-none-any.whl size=8312 sha256=d41b62bad5aee3dd55d8c04722b8c2d35cd3b3448f2481bf2d71d3fc6138d910
  Stored in directory: /tmp/pip-ephem-wheel-cache-ceqqjby2/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
ERROR: Could not find a version that satisfies the requirement keras_application (from versions: none)
ERROR: No matching distribution found for keras_application
Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 377.1MB 46kB/s 
     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 3.2MB 50.9MB/s 
     |████████████████████████████████| 491kB 54.1MB/s 
  Found existing installation

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

Mounted at /content/drive


In [ ]:
def add_top_layer(model_base,output_classes):
  bottom_model = model_base.output
  """creates the top or head of the model that will be
  placed ontop of the bottom layers"""
  last_layer = model_base.get_layer('avg_pool').output
  bottom_model = Flatten(name="flatten")(last_layer)
  bottom_model = Dense(1024,activation='relu')(bottom_model)
  bottom_model = Dropout(0.50)(bottom_model)
  bottom_model = Dense(1024,activation='relu')(bottom_model)
  bottom_model = Dropout(0.50)(bottom_model)
  bottom_model = Dense(output_classes,activation='sigmoid')(bottom_model)
  return bottom_model
print("complete")

complete


In [ ]:
### CCC implementations

import tensorflow.keras.backend as K

def ccc_loss(y_true, y_pred):
    return 1 - ((ccc_v(y_true, y_pred) + ccc_a(y_true, y_pred)) / 2)


def ccc_v(y_true, y_pred):
    """
    Concordance Correlation Coefficient for valence
    """
    x = y_true[:, 0]
    y = y_pred[:, 0]
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    rho = K.sum(xm * ym) / (K.sqrt(K.sum(xm ** 2)) * K.sqrt(K.sum(ym ** 2)))
    x_s = K.std(x)
    y_s = K.std(y)
    ccc = 2 * rho * x_s * y_s / (x_s ** 2 + y_s ** 2 + (mx - my) ** 2)
    return ccc


def ccc_a(y_true, y_pred):
    """
    Concordance Correlation Coefficient for arousal
    """
    x = y_true[:, 1]
    y = y_pred[:, 1]
    mx = K.mean(x, axis=0)
    my = K.mean(y, axis=0)
    xm, ym = x - mx, y - my
    rho = K.sum(xm * ym) / (K.sqrt(K.sum(xm ** 2)) * K.sqrt(K.sum(ym ** 2)))
    x_s = K.std(x)
    y_s = K.std(y)
    ccc = 2 * rho * x_s * y_s / (x_s ** 2 + y_s ** 2 + (mx - my) ** 2)
    return ccc

print('completed')

completed


In [ ]:
#Finalizing the model:
from keras_vggface.vggface import VGGFace

model_base = VGGFace(model='senet50',include_top=False, input_shape=(224, 224, 3), pooling='avg') # pooling: None, avg or max


for layers in model_base.layers:
  layers.trainable = True
## adding the regression layer on top
model_head=add_top_layer(model_base,2)
#Connecting the two models
model=Model(inputs=model_base.input,outputs=model_head)

#last_layer = model_base.get_layer('avg_pool').output
#x = Flatten(name='flatten')(last_layer)
#out = Dense(2, activation='sigmoid')(x)
#model = Model(model_base.input, out)


opt = Adam(lr=0.0001, decay=1e-6)
#Selecting loss and metrics
model.compile(optimizer=opt, loss=ccc_loss, metrics=[ccc_v, ccc_a])
#Printing model summary
print(model.summary())
print('completed')









104947712/104944616 [==============================] - 2s 0us/step
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        input_1[0][0]                    
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Act

In [ ]:
#Loading the Data_set
#mount drive:
# Change data_src variable as per your drive
data_src = '/content/drive/MyDrive/Colab/AffectNet'

#Loading the data into the correct format:
train_faces = np.load(data_src+'/training_images.npy')
#normalizing
train_annotations =(np.load(data_src+'/train_annotations.npy')+1)/2
#Only for grayscale.
#train_faces=np.reshape(train_faces,(train_faces.shape[0],train_faces.shape[1],train_faces.shape[2],1))


print(train_faces.shape)
print(train_annotations.shape)
print('completed')

(12105, 224, 224, 3)
(12105, 2)
completed


In [ ]:
## Putting the data into training classes:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(train_faces, train_annotations, test_size=0.20, shuffle= True)
print(x_train.shape)
train_faces=None
train_annotations=None
##Try it out with a larger batch size
batch_size=64
#### Fit the dataset
train_datagen = ImageDataGenerator(rescale=1./255,
                                  zoom_range=0.2,
                                  horizontal_flip=True)

training_set=train_datagen.flow(x_train,y_train,batch_size=batch_size)
x_train=None

val_datagen = ImageDataGenerator(rescale=1./255,
                                 zoom_range=0.2,
                                 horizontal_flip=True)

val_set=val_datagen.flow(x_valid,y_valid,batch_size=batch_size)
x_valid=None
print('completed')

(9684, 224, 224, 3)
completed


In [ ]:
PATH="/content/drive/MyDrive/Colab/AffectNet/"
chk_path = PATH+'model_SENet_224.h5'
log_dir = PATH+"checkpoint/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")

checkpoint = ModelCheckpoint(filepath=chk_path,
                               save_best_only=True,
                               verbose=1,
                               mode='min',
                               monitor='val_loss')

earlystop = EarlyStopping(monitor='val_loss',
                            min_delta=0,
                            patience=3,
                            verbose=1,
                            restore_best_weights=True)

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                                factor=0.2,
                                patience=6,
                                verbose=1,
                                min_delta=0.0001)

steps_per_epoch = training_set.n // training_set.batch_size
validation_steps = val_set.n // val_set.batch_size

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
csv_logger = CSVLogger(PATH+'training_SENet_224.log')

callback = [checkpoint, reduce_lr, csv_logger]
print(steps_per_epoch)
print(validation_steps)

print('completed')

151
37
completed


In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess)

In [ ]:
with tf.device('/device:GPU:0'):
  epochs=30
  model_hist = model.fit_generator(generator=training_set,
                  validation_data=val_set,
                  epochs=epochs,
                  callbacks=callback,
                  steps_per_epoch=steps_per_epoch,
                  validation_steps=validation_steps)

save_path = '/content/drive/MyDrive/Colab/afew/model_SENet_224_finished.h5'
model.save_weights(save_path)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/30
151/151 [==============================] - 317s 2s/step - loss: 0.5919 - ccc_v: 0.5713 - ccc_a: 0.2448 - val_loss: 0.7312 - val_ccc_v: 0.2996 - val_ccc_a: 0.2379

Epoch 00001: val_loss improved from inf to 0.73122, saving model to /content/drive/MyDrive/Colab/AffectNet/model_SENet_224.h5
Epoch 2/30
151/151 [==============================] - 284s 2s/step - loss: 0.4685 - ccc_v: 0.6550 - ccc_a: 0.4079 - val_loss: nan - val_ccc_v: nan - val_ccc_a: nan

Epoch 00002: val_loss did not improve from 0.73122
Epoch 3/30
151/151 [==============================] - 283s 2s/step - loss: 0.4190 - ccc_v: 0.7021 - ccc_a: 0.4600 - val_loss: 0.5140 - val_ccc_v: 0.6483 - val_ccc_a: 0.3238

Epoch 00003: val_loss improved from 0.73122 to 0.51395, saving model to /content/drive/MyDrive/Colab/AffectNet/model_SENet_224.h5
Epoch 4/30
151/151 [==============================] - 284s 2s/step - loss: 0.4570 - cc

In [ ]:
###Results for the model
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
plt.figure(figsize=(8,6))
#plt.subplot(1,2,2)
#plt.plot(model_hist.history['ccc_a'])
#plt.plot(model_hist.history['val_ccc_a'])
#plt.title('model CCC')
#plt.xlabel('Epochs')
#plt.ylabel('CCC Arousal')
#plt.legend(['train', 'test'], loc='upper left')

#plt.subplot(1,2,1)
plt.plot(model_hist.history['loss'])
plt.plot(model_hist.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epochs')
plt.ylabel('CCC Loss')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

In [ ]:
###Results for the model
plt.figure(figsize=(14,5))
plt.subplot(1,2,2)
plt.plot(model_hist.history['ccc_a'])
plt.plot(model_hist.history['val_ccc_a'])
plt.title('Model CCC Arousal')
plt.xlabel('Epochs')
plt.ylabel('CCC Arousal')
plt.legend(['train', 'test'], loc='upper left')

plt.subplot(1,2,1)
plt.plot(model_hist.history['ccc_v'])
plt.plot(model_hist.history['val_ccc_v'])
plt.title('Model CCC Valence')
plt.xlabel('Epochs')
plt.ylabel('CCC Valence')
plt.legend(['train', 'test'], loc='upper left')

plt.show()

In [ ]:
##Loading weigths 
PATH="/content/drive/MyDrive/Colab/AffectNet/"
# Load the previously saved weights
model.load_weights(PATH+'model_SENet_224_finished.h5')

#Loading the Data_set
# Change data_src variable as per your drive
data_src = '/content/drive/MyDrive/Colab/AffectNet'

#Loading the data into the correct format:
test_faces = np.load(data_src+'/validation_images.npy')/255
test_labels = (np.load(data_src+'/validation_annotations.npy')+1)/2
#Only for grayscale.
#from sklearn.model_selection import train_test_split

#x_test, _, y_test, _ = train_test_split(test_faces, test_labels, test_size=0.001, shuffle= True)
#train_datagen = ImageDataGenerator(rescale=1./255)

#training_set=train_datagen.flow(x_test,y_test,batch_size=64)
print(test_labels)

print('completed')

[[0.411577   0.4611801 ]
 [0.3161055  0.5919475 ]
 [0.46854435 0.7661635 ]
 ...
 [0.23545    0.853836  ]
 [0.3492065  0.6190475 ]
 [0.362079   0.7951995 ]]
completed


In [ ]:
#Executing tests
print("Evaluate on test data")

results = model.evaluate( x=test_faces, y=test_labels, batch_size=64)
print("test loss, test acc:", results)

Evaluate on test data
3998/3998 [==============================] - 15s 4ms/step
test loss, test acc: [0.5188820334062867, 0.5135474781384165, 0.44868845075532876]
